In [1]:
def multiply_pair(A, B):
    import numpy as np  # Only on the worker
    A_np = np.array(A, dtype=float)
    B_np = np.array(B, dtype=float)
    C_np = A_np @ B_np
    return C_np.tolist()  # convert back to nested list


In [3]:
import random
import ndcctools.taskvine as vine

N = 50      # 50x50 matrices
num_pairs = 10  # We'll do 10 pairs for demonstration

A_list = []
B_list = []

for _ in range(num_pairs):
    A_mat = [[random.random() for _ in range(N)] for _ in range(N)]
    B_mat = [[random.random() for _ in range(N)] for _ in range(N)]
    A_list.append(A_mat)
    B_list.append(B_mat)

m = vine.Manager([9123,9150], name="matrix")

print(f"[manager] Listening on port {m.port}")

tasks_map = {}
results = [None]*num_pairs

for i in range(num_pairs):
    task = vine.PythonTask(multiply_pair, A_list[i], B_list[i])
    t_id = m.submit(task)
    tasks_map[t_id] = i
    print(f"[manager] Submitted multiplication for pair index {i}.")

print("[manager] Waiting for tasks to complete...")
while not m.empty():
    done_task = m.wait(5)
    if done_task:
        idx = tasks_map[done_task.id]
        if done_task.successful():
            C = done_task.output
            results[idx] = C
            print(f"[manager] Pair {idx} -> result row0 length = {len(C[0])}")
        else:
            print(f"[manager] Task for pair {idx} failed: {done_task.result}")

print("\n[manager] All tasks done.")

/afs/crc.nd.edu/user/m/mislam5/miniconda3/envs/floability-env-2/lib/python3.13/site-packages/ndcctools/taskvine/__init__.py:79: DaskVineWarning: DaskVine not available. Couldn't find module: dask
  warnings.warn(f"DaskVine not available. Couldn't find module: {e.name}", DaskVineWarning)


[manager] Listening on port 9123
[manager] Submitted multiplication for pair index 0.
[manager] Submitted multiplication for pair index 1.
[manager] Submitted multiplication for pair index 2.
[manager] Submitted multiplication for pair index 3.
[manager] Submitted multiplication for pair index 4.
[manager] Submitted multiplication for pair index 5.
[manager] Submitted multiplication for pair index 6.
[manager] Submitted multiplication for pair index 7.
[manager] Submitted multiplication for pair index 8.
[manager] Submitted multiplication for pair index 9.
[manager] Waiting for tasks to complete...
[manager] Pair 2 -> result row0 length = 50
[manager] Pair 6 -> result row0 length = 50
[manager] Pair 4 -> result row0 length = 50
[manager] Pair 5 -> result row0 length = 50
[manager] Pair 1 -> result row0 length = 50
[manager] Pair 9 -> result row0 length = 50
[manager] Pair 7 -> result row0 length = 50
[manager] Pair 0 -> result row0 length = 50
[manager] Pair 3 -> result row0 length = 5